In [ ]:
!pip3 install Augmentor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import Augmentor
import numpy as np
import os
import glob
import random
import collections
from PIL import Image

random.seed(0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Point to a Root Directory

Your root directory must contain subdirectories, one for each class in your machine learning classification problem:

In [ ]:
root_directory = "drive/MyDrive/Ternaus/train/1"

## 2. Scan for folders in the root directory

We use `glob.glob()` to scan for all files in the `root_directory` and only choose those that are directories. These will be out classes:

In [ ]:
p = Augmentor.Pipeline(root_directory)

Initialised with 543 image(s) found.
Output directory set to drive/MyDrive/Ternaus/train/1/output.

In [ ]:
p.flip_top_bottom(probability=0.1)
p.flip_top_bottom(probability=0.4)
p.rotate(probability=0.3, max_left_rotation=5, max_right_rotation=5)
p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.1)
p.flip_left_right(probability=0.4)
p.rotate(probability=1.0, max_left_rotation=5, max_right_rotation=10)

In [ ]:
p.status()

Operations: 8
	0: Flip (probability=0.1 top_bottom_left_right=TOP_BOTTOM )
	1: Flip (probability=0.4 top_bottom_left_right=TOP_BOTTOM )
	2: RotateRange (probability=0.3 max_left_rotation=-5 max_right_rotation=5 )
	3: Zoom (probability=0.3 min_factor=1.1 max_factor=1.6 )
	4: RotateRange (probability=0.7 max_left_rotation=-10 max_right_rotation=10 )
	5: Flip (probability=0.1 top_bottom_left_right=LEFT_RIGHT )
	6: Flip (probability=0.4 top_bottom_left_right=LEFT_RIGHT )
	7: RotateRange (probability=1.0 max_left_rotation=-5 max_right_rotation=10 )
Images: 1475
Classes: 1
	Class index: 0 Class label: 8 
Dimensions: 1
	Width: 28 Height: 28
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


In [ ]:
p.sample(10000)

Processing <PIL.Image.Image image mode=L size=28x28 at 0x7F9DC4487350>: 100%|██████████| 10000/10000 [03:02<00:00, 54.93 Samples/s]


In [ ]:
i = 0
for i in range(11):
  print(i)
  root_directory = f"drive/MyDrive/Fashion MNIST/data_fashion/test/{i}"
  p = Augmentor.Pipeline(root_directory)
  p.flip_top_bottom(probability=0.1)
  p.flip_top_bottom(probability=0.4)
  p.rotate(probability=0.3, max_left_rotation=5, max_right_rotation=5)
  p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
  p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
  p.flip_left_right(probability=0.1)
  p.flip_left_right(probability=0.4)
  p.rotate(probability=1.0, max_left_rotation=5, max_right_rotation=10)
  p.sample(12000)

0
Initialised with 1000 image(s) found.
Output directory set to drive/MyDrive/Fashion MNIST/data_fashion/test/0/output.

Processing <PIL.Image.Image image mode=L size=28x28 at 0x7F4CD99B7110>:   8%|▊         | 981/12000 [00:17<03:16, 56.19 Samples/s]


KeyboardInterrupt: ignored

In [ ]:
root_directory = "drive/MyDrive/Fashion MNIST/data_fashion/train/1/output"
p = Augmentor.Pipeline(root_directory)

Initialised with 10000 image(s) found.
Output directory set to drive/MyDrive/Fashion MNIST/data_fashion/train/1/output/output.

In [ ]:
folders = []
for f in glob.glob(root_directory):
    if os.path.isdir(f):
        folders.append(os.path.abspath(f))

print("Folders (classes) found: %s " % [os.path.split(x)[1] for x in folders])

Folders (classes) found: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 


## 3. Create a pipeline for each class

Now we create a pipeline object for each of the classes. MNIST consists of 10 digits, and each digit represents one class:

In [ ]:
pipelines = {}
for folder in folders:
    print("Folder %s:" % (folder))
    pipelines[os.path.split(folder)[1]] = (Augmentor.Pipeline(folder))
    print("\n----------------------------\n")

Folder /content/drive/MyDrive/Fashion MNIST/data_fashion/train/0:
Initialised with 1425 image(s) found.
Output directory set to /content/drive/MyDrive/Fashion MNIST/data_fashion/train/0/output.
----------------------------

Folder /content/drive/MyDrive/Fashion MNIST/data_fashion/train/1:
Initialised with 1525 image(s) found.
Output directory set to /content/drive/MyDrive/Fashion MNIST/data_fashion/train/1/output.
----------------------------

Folder /content/drive/MyDrive/Fashion MNIST/data_fashion/train/2:
Initialised with 1475 image(s) found.
Output directory set to /content/drive/MyDrive/Fashion MNIST/data_fashion/train/2/output.
----------------------------

Folder /content/drive/MyDrive/Fashion MNIST/data_fashion/train/3:
Initialised with 1485 image(s) found.
Output directory set to /content/drive/MyDrive/Fashion MNIST/data_fashion/train/3/output.
----------------------------

Folder /content/drive/MyDrive/Fashion MNIST/data_fashion/train/4:
Initialised with 1470 image(s) found.


We can summarise what was scanned:

In [ ]:
for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))

Class 0 has 1425 samples.
Class 1 has 1525 samples.
Class 2 has 1475 samples.
Class 3 has 1485 samples.
Class 4 has 1470 samples.
Class 5 has 1479 samples.
Class 6 has 1536 samples.
Class 7 has 1475 samples.
Class 8 has 1475 samples.
Class 9 has 1521 samples.


## 4. Add operations to the pipelines

Here we will add operations to each of the pipelines. Some operations will be applied to all pipelines, others only to some pipelines.

Here we add a rotate operation to all pipelines (and hence will be applied to all digits):

In [ ]:
pipelines

{'0': <Augmentor.Pipeline.Pipeline at 0x7f797c1f4c10>,
 '1': <Augmentor.Pipeline.Pipeline at 0x7f797c0db9d0>,
 '2': <Augmentor.Pipeline.Pipeline at 0x7f797bf74690>,
 '3': <Augmentor.Pipeline.Pipeline at 0x7f797bed1110>,
 '4': <Augmentor.Pipeline.Pipeline at 0x7f7982e6e090>,
 '5': <Augmentor.Pipeline.Pipeline at 0x7f797bd11cd0>,
 '6': <Augmentor.Pipeline.Pipeline at 0x7f797bbf1cd0>,
 '7': <Augmentor.Pipeline.Pipeline at 0x7f797bbf1f50>,
 '8': <Augmentor.Pipeline.Pipeline at 0x7f797ba3a510>,
 '9': <Augmentor.Pipeline.Pipeline at 0x7f797b999710>}

In [ ]:
for pipeline in pipelines.values():
    pipeline.rotate(probability=0.5, max_left_rotation=5, max_right_rotation=5)

Here we add some operations that we only want to apply to certain classes. The figure 8 can be flipped horizontally and vertically:

In [ ]:
pipelines["8"].flip_top_bottom(probability=0.5)
pipelines["8"].flip_left_right(probability=0.5)

While the figure 3 can only be flipped vertically:

In [ ]:
pipelines["3"].flip_top_bottom(probability=0.5)

## 5. Define a class label / class integer map

The classes will have string labels associated with them, depending on the name of each class's parent folder. Here you must map the names of each of your classes with the 0-based index (which must correspond to the test data of your dataset).

In the case of MNIST this is quite easy, the samples for the digit 0 were stored in a folder 0 and have the text label 0, and so on:

In [ ]:
integer_labels = {'0': 0, 
                  '1': 1, 
                  '2': 2, 
                  '3': 3, 
                  '4': 4, 
                  '5': 5, 
                  '6': 6, 
                  '7': 7, 
                  '8': 8, 
                  '9': 9,
                  '10': 10}

## 6. Define pipeline containers to store the pipelines and additional information

Later we will need each pipeline's 0-based integer label as well as its categorical label (depending on the type of neural network you define):

In [ ]:
PipelineContainer = collections.namedtuple('PipelineContainer', 
                                           'label label_integer label_categorical pipeline generator')

pipeline_containers = []

for label, pipeline in pipelines.items():
    label_categorical = np.zeros(len(pipelines), dtype=np.int)
    label_categorical[integer_labels[label]] = 1
    pipeline_containers.append(PipelineContainer(label, 
                                                 integer_labels[label], 
                                                 label_categorical, 
                                                 pipeline, 
                                                 pipeline.keras_generator(batch_size=1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


## 7. Define a generator function

Neural networks in Keras can be supplied with a generator to supply training data. Because we have one generator for each pipeline, we need to create "generator of generators":

In [ ]:
def multi_generator(pipeline_containers, batch_size):
    while True:
        X = []
        y = []
        for i in range(batch_size):
            pipeline_container = random.choice(pipeline_containers)
            image, _ = next(pipeline_container.generator)
            image = image.reshape((28,28,1)) # Or (1, 28, 28) for channels_first, see Keras' docs.
            X.append(image)
            y.append(pipeline_container.label_categorical)  # Or label_integer if required by network
        X = np.asarray(X)
        y = np.asarray(y)
        yield X, y

In [ ]:
for p in pipelines.values():
    print("Class %s has %s samples." % (p.augmentor_images[0].class_label, len(p.augmentor_images)))

Class 0 has 1425 samples.
Class 1 has 1525 samples.
Class 2 has 1475 samples.
Class 3 has 1485 samples.
Class 4 has 1470 samples.
Class 5 has 1479 samples.
Class 6 has 1536 samples.
Class 7 has 1475 samples.
Class 8 has 1475 samples.
Class 9 has 1521 samples.


## 8. Create the generator object

Create a generator, `g` to pass data randomly from each pipeline (and hence each class) to a neural network:

In [ ]:
batch_size = 128

g = multi_generator(pipeline_containers=pipeline_containers, 
                    batch_size=batch_size)  # Here the batch size can be set to any value

To generate a batch of 128 images and labels, at random from a random pipeline defined above, we can use the `next()` function:

In [ ]:
X, y = next(g)

In [ ]:
X

array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.00784314],
         [0.        ],
         [0.01960784],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.01176471],
         [0.00392157],
         [0.01568628],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.        ],
         [0.00392157],
         [0.00

In [ ]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0]])

We can confirm that we are receiving images in batches of 128 and that the labels correspond to the images in each pipeline:

We can use PIL to view the augmented images and cofirm the labels match (note that PIL requires images to be specified differently to how Keras expects data, hence some preprocessing of the data must be performed):

In [ ]:
image_index = 8  # Take image index 3 from the batch

x_array = X[image_index]
x_array = x_array.reshape((28,28))
x_array = x_array * 255
x_array = x_array.astype(np.uint8)
Image.fromarray(x_array)

The label below should correspond to the image output above:

In [ ]:
print("Image label: %s" % (np.nonzero(y[image_index])[0][0]))

Image label: 6


In [ ]:
pipeline_containers[0][1]

0

## 9. Train a neural network with the generator

Last, we train a neural network with the differing pipelines for each class.

First we define a neural network:

In [ ]:
num_classes = len(pipelines)
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Once a network has been defined, you can compile it so that the model is ready to be trained with data:

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Using the same batch size as the generator above, we can begin to train the neural network:

In [ ]:
h = model.fit_generator(g, steps_per_epoch=50000/batch_size, epochs=10, verbose=1)

Epoch 1/10
390/390 [==============================] - 31s - loss: 0.4767 - acc: 0.8502    
Epoch 2/10
390/390 [==============================] - 31s - loss: 0.1364 - acc: 0.9595    
Epoch 3/10
390/390 [==============================] - 30s - loss: 0.1029 - acc: 0.9691    
Epoch 4/10
390/390 [==============================] - 30s - loss: 0.0879 - acc: 0.9740    
Epoch 5/10
390/390 [==============================] - 30s - loss: 0.0778 - acc: 0.9771    
Epoch 6/10
390/390 [==============================] - 30s - loss: 0.0689 - acc: 0.9784    
Epoch 7/10
390/390 [==============================] - 30s - loss: 0.0650 - acc: 0.9804    
Epoch 8/10
390/390 [==============================] - 31s - loss: 0.0632 - acc: 0.9808    
Epoch 9/10
390/390 [==============================] - 30s - loss: 0.0594 - acc: 0.9828    
Epoch 10/10
390/390 [==============================] - 30s - loss: 0.0531 - acc: 0.9845    
